In [1]:
SHORTEN = {
    "Artery-Aorta":"Artery.A"     ,
    "Artery-Tibial": "Artery.T",
    "Adipose-Subcutaneous": "Adipose S.",    
    "Adipose-Visceral(Omentum)":"Adipose V.",
    "Brain-Caudate(basalganglia)":"B.Caudate"   , 
    "Brain-Cerebellum":"B.Cerebellum",
    "Cells-Transformedfibroblasts": "Fibroblast",
    "Esophagus-Mucosa": "E.Mucosa",
    "Esophagus-Muscularis":"E.Muscularis",
    "Heart-LeftVentricle":"Ventricule",
    "Lung": "Lung",
    "Muscle-Skeletal": "Muscle",
    "Nerve-Tibial":"Nerve",
    "Skin-NotSunExposed(Suprapubic)": "Skin.Unexposed",
    "Skin-SunExposed(Lowerleg)":"Skin.Leg",
    "Thyroid":"Thyroid",
    "WholeBlood": "Blood",
}

import math
import numpy as np
import pandas as pd
from collections import defaultdict
import subprocess

tissu = sorted([x for x in SHORTEN.keys()])
CHR = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,'X']
Gene_table = pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/FEATURES/Genes_only_table', sep='\t')
Motifs = pd.read_csv('/storage/resources/dbase/human/hg19/hg19.hipstr_reference_withmotif.bed', sep='\t')

In [2]:
Table0 = defaultdict(list)
for Tissue in tissu:
    print(Tissue)

    LR = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/PQValues'
    ANOV = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/HH/anova_wg.csv'
    CAUSE = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/HH/caviar.out'
    samples = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/Corr_Expr.csv'

    anova = pd.read_csv(ANOV, sep=',')
    anova = anova[['chrom','gene','str.start','anova_pval','estr_fdr','esnp_fdr','delta_bic','delta_aic']]
    lr = pd.read_csv(LR,sep='\t')
    Cs=[]
    for i in CHR:
        C=pd.read_csv(CAUSE+str(i), sep='\t')
        Cs.append(C)
    causal=Cs[0]
    for i in range(1,23,1):
        causal=causal.append(Cs[i])   
    causal['best.str.start'] = causal['best.str.start'].astype(int)
    caviarbest=[x for x in list(causal['top.variant']) if 'STR_' in x]
    estrs = lr.loc[lr['qvalue']<=0.1]
    Table0['Num.genes'].append(lr.shape[0])
    Table0['Num.estrs'].append(estrs.shape[0])
    anovaP = anova.loc[anova['anova_pval']<=0.05]  #anova pass
    anovaPe = anovaP.loc[anovaP['gene'].isin(list(estrs['gene']))]  #estrs + anova pass
    anovaSTR = anova.loc[(anova['delta_aic']>0) & (anova['delta_bic']>0)] #snp+str prefered
    X = anovaSTR.loc[anovaSTR['gene'].isin(list(anovaPe['gene']))]  #estrs + snp+STR + pass
    Table0['ANOVA.pass'].append(anovaPe.shape[0])
    Table0['Num.BestCaviar'].append(len(caviarbest))
    cmd = len(open(samples,'r').readlines())
    Table0['Samples'].append (int(cmd)-1)
    Table0['ANOVA.STR+SNP'].append(anovaSTR.shape[0])
    Table0['eSTR+SNP.anova.pass'].append(X.shape[0])
    Table0['e.ANOVA.CAVIAR'].append(X.loc[X['gene'].isin(list(causal.loc[causal['top.variant'].isin(caviarbest)]['gene']))].shape[0])
    Table0['eSTR.CAVIAR'].append(estrs.loc[estrs['gene'].isin(list(causal.loc[causal['top.variant'].isin(caviarbest)]['gene']))].shape[0])

 
Table0['Tissues']=tissu
Table = pd.DataFrame(Table0)
Table.index =[SHORTEN[x] for x in list( Table['Tissues'])]
del Table['Tissues']
Table = Table[['Samples','Num.genes','Num.estrs','Num.BestCaviar','eSTR.CAVIAR','ANOVA.pass','ANOVA.STR+SNP','eSTR+SNP.anova.pass','e.ANOVA.CAVIAR']]
print('END!')

Table.sort_values("e.ANOVA.CAVIAR", ascending=False)

Adipose-Subcutaneous
Adipose-Visceral(Omentum)
Artery-Aorta
Artery-Tibial
Brain-Caudate(basalganglia)
Brain-Cerebellum
Cells-Transformedfibroblasts
Esophagus-Mucosa
Esophagus-Muscularis
Heart-LeftVentricle
Lung
Muscle-Skeletal
Nerve-Tibial
Skin-NotSunExposed(Suprapubic)
Skin-SunExposed(Lowerleg)
Thyroid
WholeBlood
END!


,Samples,Num.genes,Num.estrs,Num.BestCaviar,eSTR.CAVIAR,ANOVA.pass,ANOVA.STR+SNP,eSTR+SNP.anova.pass,e.ANOVA.CAVIAR
Nerve,265,12567,1681,258,111,407,690,276,56
E.Muscularis,214,12432,1028,220,83,255,611,187,52
Adipose S.,270,12432,1353,248,105,304,573,206,49
Thyroid,262,12616,1523,249,99,367,647,251,45
Artery.T,276,12140,1257,242,102,315,562,220,44
Muscle,343,11970,1273,245,98,301,442,178,43
E.Mucosa,255,12474,1228,244,91,276,563,190,41
Adipose V.,193,12528,693,288,81,157,560,111,38
Skin.Leg,297,12606,1462,233,106,321,514,198,36
Fibroblast,225,11897,1108,245,80,239,539,161,32


In [7]:
#q = pd.read_csv('/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/LR_SummaryTest_Table.tsv',sep='\t')
#q.loc[q['gene']=='ENSG00000160213.5']
#100*149/2436
X.loc[X['gene'].isin(list(causal.loc[causal['top.variant'].isin(caviarbest)]['gene']))]

,chrom,gene,str.start,anova_pval,estr_fdr,esnp_fdr,delta_bic,delta_aic
459,chr1,ENSG00000134247.9,117556808,0.004327,0.087358,1.000000,2.589801,6.242290
582,chr1,ENSG00000084636.13,32044259,0.000474,0.083936,1.000000,6.798188,10.377918
918,chr1,ENSG00000197965.7,167737092,0.001980,0.016046,1.000000,3.947508,7.677608
976,chr1,ENSG00000135750.10,233797970,0.008308,0.010981,0.615988,1.291539,5.044112
1814,chr2,ENSG00000162999.8,184035958,0.005452,0.029353,1.000000,2.013252,5.806266
2211,chr3,ENSG00000174564.8,136642402,0.000570,0.023987,1.000000,6.243729,10.002631
2769,chr4,ENSG00000163138.14,20607320,0.000522,0.023552,1.000000,6.514626,10.181053
4131,chr6,ENSG00000196230.8,30704959,0.002483,0.016415,1.000000,3.721534,7.278362
4959,chr8,ENSG00000181195.6,57438742,0.005048,0.077194,1.000000,2.311231,5.960205
5611,chr10,ENSG00000148848.10,127831854,0.000120,0.009128,0.905767,9.196705,12.961896


In [ ]:
Number of egenes...  (2436)
Nbr of eGenes in more than 1 tissues  (1029)
egenes in at least 10 tissues  (149)
